# Elementary Cellular Automata

Cellular automata (CA) are interacting systems on a grid, whose (finite number of) future states depend on the states of the neighboring cells according to pre-defined rules.

The simplest such system is one on a 1D grid with two available states ("dead" -> 0 or "alive" -> 1). An initial configuration is selected by specifying the state of all grid cells, e.g.:

![cells](https://drive.google.com/uc?id=1Tk7475unfbg4ngirrVEF7OTwuxb6_W2_)

Let's make a random initial configuration on a 10-cell grid:

In [ ]:
configuration[0]=RandomChoice[{0,1},10]

We'll use `ArrayPlot` to visualize the configurations.
`ArrayPlot` takes an array of "pixel" values and displays them on the screen.

In [ ]:
ArrayPlot[{configuration[0]},Mesh->All]

<div class="alert alert-block alert-warning">
<b>Note: </b>
We added the option Mesh->All to differentiate between adjacent cells of the same value.
</div>

Subsequent iterations are created by iteratively applying a set of deterministic **rules**. To do so, we need to define a **neighborhood** for the rules to act on.

Let's start simple, we could only consider a cell's 'neighborhood' as being itself. Using this 'neighborhood' we could define a few rules:
- Trivial rule: each cell stays the same  
$\qquad \mathrm{configuration}[t+1][i] = \mathrm{configuration}[t][i]$

In [ ]:
trivialConfiguration[t_]:=configuration[0]
ArrayPlot[{trivialConfiguration[0], trivialConfiguration[1], 
    trivialConfiguration[2],trivialConfiguration[3]}, Mesh -> All]

- Epileptic rule: each cell 'flips' state  
$\qquad \mathrm{configuration}[t+1][i] = 1-\mathrm{configuration}[t][i]$

In [ ]:
epilepticConfiguration[t_]:=1-epilepticConfiguration[t-1]
epilepticConfiguration[0]=configuration[0];
ArrayPlot[{epilepticConfiguration[0], epilepticConfiguration[1], 
    epilepticConfiguration[2],epilepticConfiguration[3]}, Mesh -> All]

<div class="alert alert-block alert-warning">
<b>Note: </b>
We defined a recursive function, and as such we need to specify an initial state to break the recursion.
</div>

We can get much more complex behavior by defining a slightly more elaborate neighborhood.
For example, we'll consider a neighborhood with three neighbors for each cell, the first neighbor to the left (L), its self (C), and the first neighbor to the right (R).

![triplets](https://drive.google.com/uc?id=13mAjRgJ3nBcVa_8qosi-J_UvozXH98OP)

This binary triplet, such as 011 in the example shown, can be thought of as a unique identifier for that specific neighbor state configuration. We can also convert the representation of the code from binary to decimal.

If our codes are only 3 digits long in binary, how many possible combinations are there? There are $2\times2\times2=8$ possibilities. Thus, our rules for the evolution of cellular automata must prescribe an outcome for each of the 8 possible configurations we can observe. These rules were formalized and extensively studied by Wolfram. Each rule is expressed as an 8-bit binary string, such as the example Rule 30 below:

![rules](https://drive.google.com/uc?id=14v1PEupPk9eOWWUNKzG2BUktb2WN2S48)

For each of the 8 neighbor configurations, the rule defines the state of the **center cell** in the next time step.

Below, is an example `RulePlot` of one such programs - called Rule 90:

In [ ]:
RulePlot[CellularAutomaton[90]]

For example, if the center cell is "dead"(0), it's right neighbor is "alive"(1), and its left neighbor is "alive"(1) - we have to look at the third permutation rule above, which instructs us that the center cell "remains dead" (0 ->0)

In [ ]:
ArrayPlot[CellularAutomaton[90, {{1}, 0}, 127],Frame->False]

Ring a bell? We've seen this a lot these past couple of days!  
Rule 90 is one particular set of rules. Since we need to specify the outcome ("dead" or "alive") for each of these $2^3=8$ configurations, the number of distinct 1D elementary CAs is given by $2^{2^3}=256$ rules!

### Higher-Order CAs
So far, we've looked at 1D CAs which only considered the previous configuration in their update rules. We can generalize this a great number of ways. In-fact, the `CellularAutomaton` can be called in a multitude of forms:

In [ ]:
?CellularAutomaton

If you're interested, [the documentation page](https://reference.wolfram.com/language/ref/CellularAutomaton.html) for `CellularAutomaton` is rather extensive.

For now, we'll quickly demonstrate 2nd-order 1D CAs, and then switch to 2D CAs for the rest of the course! 
2nd-order CAs update a cell's state using rules which consider, not only the cell's state and the previous iteration and the cell's neighbors at the previous iteration, but also using the cell's state two iterations ago:
![second-order-CA](https://upload.wikimedia.org/wikipedia/commons/0/0f/SecondOrderCADiagram.png)

Let's start by considering Rule 150:

In [ ]:
RulePlot[CellularAutomaton[150]]

We can define this rule using a function which does the following:
- Sums up the values of all neighbors
- Takes the modulus of the sum modulo 2

In [ ]:
(*The 8 elementary configurations*)
Reverse[Tuples[{0, 1}, 3]]

In [ ]:
#->Mod[Total[Flatten[#]], 2] & /@ Reverse[Tuples[{0, 1}, 3]]
RulePlot[CellularAutomaton[150]]

<div class="alert alert-block alert-warning">
<b>Note: </b>
We used a `pure` function to define our rule above using `#`
</div>

We can then use the function-form of `CellularAutomaton` to plot a time-series starting from a single "alive" seed.  

In this form, we need to call the function as:  
`CellularAutomaton[{function,{},neighbors,order},initial-state,time-steps]`  

E.g. In evaluating the time-series on a LCR neighborhood for 1st-order Rule 150, starting from a single "alive" (1) cell in a sea of "dead" (0) cells, for 127 time-steps we would do:

In [ ]:
ArrayPlot[

 CellularAutomaton[
 (*function*)
 {Mod[Total[Flatten[#]], 2] &, {},
 
 (*neighbors - note the first index is iteration number*)
 {{0, -1}, {0, 0}, {0, 1}},
 
 (*order*)
 1},
 
 (*initial-state*)
 {{{1}}, 0},
 
 (*time-steps*)
 127],
 
  Frame->False]

In this form, it's easy to generalize our CA to a second-order CA, to obtain for example the more visually-appealing second-order Rule 150 (or rule 150R)

In [ ]:
ArrayPlot[
 CellularAutomaton[{Mod[Total[Flatten[#]], 2] &, {}, 
 {{-1, 0}, {0, -1}, {0, 0}, {0, 1}}, 2}, {{{1}, {1}}, 0}, 127],
  Frame->False]

## Two-dimensional CAs

Cellular automata were first conceived in the 1940s by Stanislaw Ulam and John Von Neumann at Los Alamos National Laboratory. It wasn't however until the 1970s and Conway's Game of Life, a two-dimensional automaton, that they became a subject of interest.

Here's a high-level implementation of Conway's Game of Life:

In [ ]:
gameOfLife = {224, {2, {{2, 2, 2}, {2, 1, 2}, {2, 2, 2}}}, {1, 1}};
board = RandomInteger[1, {50, 50}];
frames=ArrayPlot /@ CellularAutomaton[gameOfLife, board, 500];
Export["gifs/game-of-life-animation.gif", frames,ImageSize->300]

![game of life](gifs/game-of-life-animation.gif "game of life")

Let's break it down:
- It's Cellular Automaton Rule 224
- With two states: "alive"(1), "dead" (0)
- on a 3x3 'Moore' neighborhood
- totalistic rule weighting neighbors twice as much as 'self'

### 2D Neighborhoods

We've introduced one particular 2D neighborhood above, called a 'Moore' neighborhood. Neighborhoods can have a particular range, the most common being the 3x3 or 9-cell Moore neighborhood. Another common such neighborhood is the 5-cell, or 'von Neumann' neighboorhood. 
![neighborhoods](https://drive.google.com/uc?id=1sPTTDRFi-oFaw_QgH1uz6Ij4shNg_kXL)

This means that each neighbor configuration will be represented by a 5(9)-digit binary code instead of a 3-digit code, for a total of $2^5=32\;(2^9=512)$ unique configurations. As a result, our rules will be represented by 32 (512) digits instead of 8. In 2D using a von Neumann (Moore) neighborhood, there are $2^{32}=4294967296\;(2^{512}\approx 1.3\mathrm{e}{154})$ possible rule plots!

<div class="alert alert-block alert-warning">
<b>Note: </b>
The alternative python-kernel notebook illustrates how to implement a `von Neumann` neighborhood using a convolution approach. In this notebook, we'll instead demonstrate how one can leverage vectorization to implement a `Moore` neighborhood.
</div>

A number of languages' paradigm is in-line with procedural programming.  
Not so much with Mathematica, we'll leverage vectorization to avoid iterating through columns and rows (e.g. using a nested for loop). 

To do-so, we'll think about shifting our grid according to our neighborhood and operating on the entire grid simultaneously. 

We'll use the function `RotateRight`. In one-dimension the function "shifts" a list periodically by an increment

In [ ]:
Range[10]
RotateRight[Range[10], 1]
RotateRight[Range[10], 2]

In two dimensions, the function shifts an array periodically by a vector increment

In [ ]:
Array[Subscript[x, #1, #2] &, {3, 3}] // Grid

In [ ]:
RotateRight[Array[Subscript[x, #1, #2] &, {3, 3}], {0, 1}] // Grid

Let's try this for our CAs.  
Say for example our 'neighborhood' consisted of the center cell and the nearest-neighbor to the right.

In [ ]:
test = ReplacePart[
   ConstantArray[0, {3, 3}], {{2, 2} -> 1, {2, 3} -> 0.5}];
one = ArrayPlot[test, Mesh -> All, ImageSize -> Small, 
  Epilog -> {Red, 
    MapIndexed[Text[Style[#1, 24], Reverse[#2 - 1/2]] &, 
     Reverse[Partition[Range[9], 3]], {2}]}]

We're looking for an algorithm to do the following:
- At cell 4: Do computation according to rule on right neighbor
- At cell 5: Do computation according to rule on self and right neighbor
- At cell 6: Do computation according to rule on self

Instead of doing a nested-loop, we instead 'shift' the lattice by one step to the left and use vectorization:

In [ ]:
two = ArrayPlot[RotateRight[test, {0, -1}], Mesh -> All, 
  ImageSize -> Small, 
  Epilog -> {Red, 
    MapIndexed[Text[Style[#1, 24], Reverse[#2 - 1/2]] &, 
     Reverse[Partition[Range[9], 3]], {2}]}]

Notice how, when we act on these two grids, we perform computations on 4,5 and 6 as requested:

In [ ]:
GraphicsRow[{one, two}]

Let's generalize this to a Moore neighborhood!
The following function rotates the lattice by the nine neighbors, and then applies function `func` to each of those:

In [ ]:
  Moore[func__, lat_] :=   
     MapThread[func, 
       Map[RotateRight[lat, #] &, 
         {{0, 0}, {1, 0}, {0, -1}, {-1, 0}, 
           {0, 1}, {1,  -1}, {-1, -1}, {-1, 1}, 
           {1, 1}}], 2];

In [ ]:
GraphicsRow[{ArrayPlot[testConfig = RandomInteger[{0, 1}, {10, 10}], 
   Mesh -> All, 
   Epilog -> {Red, 
     MapIndexed[Text[Style[#1, 20], Reverse[#2 - 1/2]] &, 
      Reverse[testConfig], {2}]}], 
  ArrayPlot[newTestConfig = Moore[Plus, testConfig], Mesh -> All, 
   Epilog -> {Red, 
     MapIndexed[Text[Style[#1, 20], Reverse[#2 - 1/2]] &, 
      Reverse[newTestConfig], {2}]}]}]

Try and verify the above yourself.
Pick a cell on the left and count the number of black neighbors.  
Does that number match the number on the right?